In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sentence_transformers import SentenceTransformer

In [4]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [5]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

In [9]:
sentence_embeddings.shape

(3, 768)

In [11]:
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding[:5])
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [ 0.19731508  0.1210537  -0.2581328   0.02697735 -0.17997532]

Sentence: Sentences are passed as a list of string.
Embedding: [-0.257479    0.2462943   0.0946965   0.30377313  0.15252738]

Sentence: The quick brown fox jumps over the lazy dog.
Embedding: [ 0.05094927  0.24235828  0.15774007 -0.2979846   0.04995703]



In [13]:
model = SentenceTransformer('stsb-roberta-large')

100%|██████████| 1.31G/1.31G [03:32<00:00, 6.19MB/s]
